In [2]:
import pandas as pd
import numpy as np
import os
import glob
from power_to_temperature import *
from test import *

In [8]:
freqs,vals,bwidth = bandwidth(50.,90.)
Freqs = np.linspace(1e-22,250,32769)
eta_nu = eta(Freqs)
mask = (Freqs>=50)&(Freqs<=91)

In [49]:
folders = glob.glob('Data/*')
folders.sort()
i=-1
for subdirs, dirs, files in os.walk('Data'):
    dirs.sort()
    files.sort()
    short,antenna,measure = [],[],[]
    short_date,antenna_date,measure_date=[],[],[]
    for file in files:
        path = os.path.join(subdirs,file)
        date = file.split("_")[0]
        if  os.path.getsize(path)==0:
            print 'EMPTY FILE:',path
            continue
        elif file.endswith('short.dat'):
            dB_short = np.loadtxt(path,unpack=True)
            T_short = Res2Temp(dB_short,bwidth)
            short.append(T_short),short_date.append(date)
        elif file.endswith('noise.dat'):
            dB_noise = np.loadtxt(path,unpack=True)
        elif file.endswith('antenna.dat'):
            dB_antenna = np.loadtxt(path,unpack=True)
            dB_clean = dB_antenna - dB_short
            T_antenna = Radio_source_trans(dB_clean, Freqs, bwidth)
            T_measure = T_antenna/eta_nu - T_short
            antenna.append(T_antenna),measure.append(T_measure)
            antenna_date.append(date),measure_date.append(date)
            
    if i>=0 and antenna and short and measure:
        name = os.path.normpath(folders[0])
        name = name.split("/")[1]
        short = np.transpose(short)
        antenna = np.transpose(antenna)
        measure = np.transpose(measure)
        short_table = pd.DataFrame(short[mask], index = Freqs[mask], columns = short_date)
        short_table.to_hdf('Data_filter/short/'+name+'.hdf5','df')
        antenna_table = pd.DataFrame(antenna[mask], index = Freqs[mask], columns = antenna_date)
        antenna_table.to_hdf('Data_filter/antenna/'+name+'.hdf5','df')
        measure_table = pd.DataFrame(measure[mask], index = Freqs[mask], columns = measure_date)
        measure_table.to_hdf('Data_filter/Tmeas/'+name+'.hdf5','df')
    i+=1
    

[] []
-1 Data/2013-06-16-14
['2013-06-13-23-44-12', '2013-06-13-23-49-34', '2013-06-13-23-55-01'] ['2013-06-13-23-44-38', '2013-06-13-23-44-49', '2013-06-13-23-45-00', '2013-06-13-23-45-11', '2013-06-13-23-45-22', '2013-06-13-23-45-34', '2013-06-13-23-45-44', '2013-06-13-23-45-55', '2013-06-13-23-46-06', '2013-06-13-23-46-18', '2013-06-13-23-46-28', '2013-06-13-23-46-39', '2013-06-13-23-46-50', '2013-06-13-23-47-01', '2013-06-13-23-47-12', '2013-06-13-23-47-23', '2013-06-13-23-47-34', '2013-06-13-23-47-43', '2013-06-13-23-47-54', '2013-06-13-23-48-05', '2013-06-13-23-48-16', '2013-06-13-23-48-27', '2013-06-13-23-48-38', '2013-06-13-23-48-50', '2013-06-13-23-49-01', '2013-06-13-23-49-12', '2013-06-13-23-50-05', '2013-06-13-23-50-16', '2013-06-13-23-50-27', '2013-06-13-23-50-39', '2013-06-13-23-50-50', '2013-06-13-23-51-01', '2013-06-13-23-51-12', '2013-06-13-23-51-23', '2013-06-13-23-51-34', '2013-06-13-23-51-46', '2013-06-13-23-51-57', '2013-06-13-23-52-05', '2013-06-13-23-52-15', '201

KeyboardInterrupt: 

Object `pd.to_hdf` not found.


In [46]:
folders = glob.glob('Data/*')
folders.sort()

In [47]:
name = os.path.normpath(folders[0])
name.split("/")[1]

'2013-06-13-23'

In [51]:
data = pd.read_hdf('Data_filter/antenna/2013-06-13-23.hdf5')

In [52]:
data

,2013-06-13-23-44-38,2013-06-13-23-44-49,2013-06-13-23-45-00,2013-06-13-23-45-11,2013-06-13-23-45-22,2013-06-13-23-45-34,2013-06-13-23-45-44,2013-06-13-23-45-55,2013-06-13-23-46-06,2013-06-13-23-46-18,...,2013-06-13-23-58-23,2013-06-13-23-58-34,2013-06-13-23-58-43,2013-06-13-23-58-52,2013-06-13-23-59-03,2013-06-13-23-59-13,2013-06-13-23-59-23,2013-06-13-23-59-34,2013-06-13-23-59-45,2013-06-13-23-59-56
50.003052,3.128449e+26,3.223405e+26,3.200176e+26,3.176245e+26,3.142461e+26,3.156526e+26,3.211706e+26,3.110353e+26,3.213528e+26,3.105843e+26,...,3.266622e+26,3.261977e+26,3.211384e+26,3.320446e+26,3.192080e+26,3.301786e+26,3.269532e+26,3.278390e+26,3.251187e+26,3.314315e+26
50.010681,3.801877e+26,3.767350e+26,3.723569e+26,3.750670e+26,3.770959e+26,3.721392e+26,3.782517e+26,3.779276e+26,3.680957e+26,3.688414e+26,...,3.824101e+26,3.815968e+26,3.777238e+26,3.838612e+26,3.810113e+26,3.844639e+26,3.840650e+26,3.878129e+26,3.850898e+26,3.852184e+26
50.018311,3.910909e+26,3.762366e+26,3.828749e+26,3.829052e+26,3.771829e+26,3.779105e+26,3.777267e+26,3.829348e+26,3.702575e+26,3.795832e+26,...,3.758350e+26,3.785856e+26,3.836020e+26,3.811823e+26,3.872984e+26,3.859394e+26,3.844122e+26,3.909446e+26,3.803295e+26,3.862412e+26
50.025940,3.877930e+26,3.883841e+26,3.898880e+26,3.827608e+26,3.821360e+26,3.882306e+26,3.882360e+26,3.801026e+26,3.884291e+26,3.833449e+26,...,3.747507e+26,3.755653e+26,3.831227e+26,3.834196e+26,3.769202e+26,3.715772e+26,3.753825e+26,3.791583e+26,3.754907e+26,3.823925e+26
50.033569,3.919635e+26,3.865132e+26,3.846639e+26,3.791345e+26,3.845632e+26,3.884948e+26,3.919002e+26,3.736354e+26,3.900039e+26,3.795957e+26,...,3.665753e+26,3.683414e+26,3.834185e+26,3.720954e+26,3.629727e+26,3.632744e+26,3.690313e+26,3.632393e+26,3.655412e+26,3.755199e+26
50.041199,3.875189e+26,3.739837e+26,3.827767e+26,3.721780e+26,3.749553e+26,3.793347e+26,3.768446e+26,3.696836e+26,3.796227e+26,3.824071e+26,...,3.650224e+26,3.712606e+26,3.742873e+26,3.725338e+26,3.680539e+26,3.742841e+26,3.876552e+26,3.761521e+26,3.713206e+26,3.678626e+26
50.048828,3.729179e+26,3.720372e+26,3.813709e+26,3.758842e+26,3.667369e+26,3.757409e+26,3.726278e+26,3.671024e+26,3.678584e+26,3.734568e+26,...,3.772217e+26,3.698528e+26,3.693340e+26,3.713724e+26,3.758565e+26,3.795103e+26,3.898065e+26,3.769594e+26,3.878700e+26,3.627112e+26
50.056458,3.640138e+26,3.700091e+26,3.683753e+26,3.677475e+26,3.640592e+26,3.706714e+26,3.666631e+26,3.596482e+26,3.554362e+26,3.611608e+26,...,3.793387e+26,3.682078e+26,3.775993e+26,3.759532e+26,3.763160e+26,3.712566e+26,3.718565e+26,3.752248e+26,3.779803e+26,3.657715e+26
50.064087,3.631703e+26,3.728982e+26,3.731433e+26,3.653293e+26,3.677575e+26,3.681130e+26,3.751440e+26,3.681143e+26,3.586303e+26,3.731793e+26,...,3.822764e+26,3.933899e+26,3.858984e+26,3.895101e+26,3.907953e+26,3.800280e+26,3.861262e+26,3.900607e+26,3.824651e+26,3.804982e+26
50.071716,3.757775e+26,3.715646e+26,3.771538e+26,3.793333e+26,3.749164e+26,3.732282e+26,3.811294e+26,3.713016e+26,3.715352e+26,3.801351e+26,...,3.842913e+26,3.927592e+26,3.772300e+26,3.856520e+26,3.882487e+26,3.842569e+26,3.911576e+26,3.802052e+26,3.929186e+26,3.888098e+26
